In [1]:
import json
from pymongo import MongoClient
import random 
from datetime import datetime
from bson import ObjectId
import requests
import pandas as pd

In [87]:
dbName = "dbChat"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()
#cur = db.list_collections()


mongodb://localhost/dbChat


In [3]:
path='../quotes-100-en.json'
with open(path,'r') as file:
    data=json.load(file)



In [4]:
users=list(data.keys())
messages=list(data.values())
chat_ids=[1,2,3,4,5,6,7,8,9]

In [ ]:
'''#build the list of dictionaries for each message
list_of_messages=[]
for i,message in enumerate(messages):
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'user_id':random.choice(users),
         'text':message,
         'time':dt_string,
         'chat_id':[],
    
     }
    list_of_messages.append(dic)
#messages_ids=db.messages.insert_many(list_of_messages)'''

In [73]:
#build the list of dictionaries for users
list_of_users=[]
for i,user in enumerate(users):
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'user_name':random.choice(users),
         'insertion_date':dt_string,
         'chats_list':[]
    
         }
    list_of_users.append(dic)
users_ids=db.users.insert_many(list_of_users)

In [74]:
#build the list of dictionaries for chats
list_of_chats=[]
for chat in chat_ids:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'chat_name':random.choice(chat_ids),
         'creation_date':dt_string,
         'users_list':[random.choice(list(users_ids.inserted_ids)) for i in range(50)],
         'messages_list':[]
    
     }
    list_of_chats.append(dic)
    
chats_ids=db.chats.insert_many(list_of_chats)

In [75]:
##populate the database with the messages with the api

messages_ids=[]
for chat_id in chats_ids.inserted_ids:
    for i in range(90):#add 10 messages
        chat_id=str(chat_id)
        user=random.choice(list(users_ids.inserted_ids))
        #add the user to the chat
        r = requests.get(f'http://localhost:3500/chat/{chat_id}/adduser?user_id={user}')
       
        text=random.choice(messages)
        r = requests.get(f'http://localhost:3500/chat/{chat_id}/addmessage?user_id={user}&text={text}')
        
        messages_ids.append(r)
        

In [76]:
chats_ids.inserted_ids

[ObjectId('5ead3bec1f313a96eed41de6'),
 ObjectId('5ead3bec1f313a96eed41de7'),
 ObjectId('5ead3bec1f313a96eed41de8'),
 ObjectId('5ead3bec1f313a96eed41de9'),
 ObjectId('5ead3bec1f313a96eed41dea'),
 ObjectId('5ead3bec1f313a96eed41deb'),
 ObjectId('5ead3bec1f313a96eed41dec'),
 ObjectId('5ead3bec1f313a96eed41ded'),
 ObjectId('5ead3bec1f313a96eed41dee')]

In [110]:
users_ids.inserted_ids


[ObjectId('5ead3bea1f313a96eed4013e'),
 ObjectId('5ead3bea1f313a96eed4013f'),
 ObjectId('5ead3bea1f313a96eed40140'),
 ObjectId('5ead3bea1f313a96eed40141'),
 ObjectId('5ead3bea1f313a96eed40142'),
 ObjectId('5ead3bea1f313a96eed40143'),
 ObjectId('5ead3bea1f313a96eed40144'),
 ObjectId('5ead3bea1f313a96eed40145'),
 ObjectId('5ead3bea1f313a96eed40146'),
 ObjectId('5ead3bea1f313a96eed40147'),
 ObjectId('5ead3bea1f313a96eed40148'),
 ObjectId('5ead3bea1f313a96eed40149'),
 ObjectId('5ead3bea1f313a96eed4014a'),
 ObjectId('5ead3bea1f313a96eed4014b'),
 ObjectId('5ead3bea1f313a96eed4014c'),
 ObjectId('5ead3bea1f313a96eed4014d'),
 ObjectId('5ead3bea1f313a96eed4014e'),
 ObjectId('5ead3bea1f313a96eed4014f'),
 ObjectId('5ead3bea1f313a96eed40150'),
 ObjectId('5ead3bea1f313a96eed40151'),
 ObjectId('5ead3bea1f313a96eed40152'),
 ObjectId('5ead3bea1f313a96eed40153'),
 ObjectId('5ead3bea1f313a96eed40154'),
 ObjectId('5ead3bea1f313a96eed40155'),
 ObjectId('5ead3bea1f313a96eed40156'),
 ObjectId('5ead3bea1f313a

# Nltk analysis

In [ ]:
!pip3 install nltk

In [78]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/silviaserafini/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [79]:
sia = SentimentIntensityAnalyzer()

In [80]:
import ast#allthe messages in all the chats
chat_ids=chats_ids.inserted_ids
messages1={}
for chat_id in chats_ids.inserted_ids:
    r = requests.get(f'http://localhost:3500//chat/{str(chat_id)}/list')
    messages1.update(r.json())
    

users_messages={}
#prendo tutti i messaggi di un utente e li unisco
for k,v in messages1.items():
    mes_user=db.messages.find_one({'_id':ObjectId(k)})#fetcg=h the message
    user=mes_user['user_id']#fetch the user
    user_texts=list(db.messages.find({"user_id":user},{'text':1}))#fetch all the messages for the user
    user_text=''
    for text in user_texts:
        try:
            text = ast.literal_eval(text['text'])
        except:
            text=list(text)
        txt=''
        for t in text:#all the messages in an unic string
            txt += t
        user_text=user_text +' '+ txt
        users_messages[user]=user_text   
    
    
sentiments={}#k =user, value={text,score}
for k,v in users_messages.items():    
    sentiments[k]={'text':str(v),"score":sia.polarity_scores(str(v))}
sent1={}
for k,v in sentiments.items():#k =user, value=text
    sent1[k]=v['text']

In [51]:
'''import ast
chat_id='5ead2b381f313a96eed3c7e7'
r = requests.get(f'http://localhost:3500//chat/{chat_id}/list')
messages1=r.json()
print(len(messages1))

users_messages={}
#prendo tutti i messaggi di un utente e li unisco
for k,v in messages1.items():
    mes_user=db.messages.find_one({'_id':ObjectId(k)})#fetcg=h the message
    user=mes_user['user_id']#fetch the user
    user_texts=list(db.messages.find({"user_id":user},{'text':1}))#fetch all the messages for the user
    user_text=''
    for text in user_texts:
        text = ast.literal_eval(text['text'])
        txt=''
        for t in text:#all the messages in an unic string
            txt += t
        user_text=user_text +' '+ txt
        users_messages[user]=user_text   
    
    
sentiments={}#k =user, value={text,score}
for k,v in users_messages.items():    
    sentiments[k]={'text':v,"score":sia.polarity_scores(v)}
sent1={}
for k,v in sentiments.items():#k =user, value=text
    sent1[k]=v['text']
'''


90


In [ ]:
'''with open('sent1.txt', 'w') as outfile:
    json.dump(sent1, outfile)'''

In [111]:
sia.polarity_scores('I do this thing called whatever I want')

{'compound': 0.0772, 'neg': 0.0, 'neu': 0.794, 'pos': 0.206}

In [81]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

sparse_matrix = count_vectorizer.fit_transform(sent1.values())

In [82]:
sparse_matrix
text_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(text_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=sent1.keys())
display(df)

,000,10,100,102,105,11,110,115,12,122,...,yun,zack,zarte,zeal,zealous,zen,zombie,zone,zoo,æsop
5ead3bea1f313a96eed41c32,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead3bea1f313a96eed414f4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead3bea1f313a96eed41ab4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead3bea1f313a96eed41bd4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead3bea1f313a96eed412d9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ead3bea1f313a96eed41d3e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead3bea1f313a96eed4017b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead3bea1f313a96eed41502,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead3bea1f313a96eed414c4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
from sklearn.metrics.pairwise import cosine_similarity as distance

similarity_matrix = distance(df,df)
print(similarity_matrix)

[[1.         0.14952043 0.07505866 ... 0.03230011 0.33706828 0.40256892]
 [0.14952043 1.         0.03984095 ... 0.15430335 0.13012001 0.27943108]
 [0.07505866 0.03984095 1.         ... 0.12909944 0.         0.02062842]
 ...
 [0.03230011 0.15430335 0.12909944 ... 1.         0.         0.07989355]
 [0.33706828 0.13012001 0.         ... 0.         1.         0.437919  ]
 [0.40256892 0.27943108 0.02062842 ... 0.07989355 0.437919   1.        ]]


In [84]:
#compute the similarity matrix
sim_df = pd.DataFrame(similarity_matrix, columns=sent1.keys(), index=sent1.keys())
display(sim_df)


,5ead3bea1f313a96eed41c32,5ead3bea1f313a96eed414f4,5ead3bea1f313a96eed41ab4,5ead3bea1f313a96eed41bd4,5ead3bea1f313a96eed412d9,5ead3bea1f313a96eed41a35,5ead3bea1f313a96eed4194a,5ead3bea1f313a96eed40dc4,5ead3bea1f313a96eed41704,5ead3bea1f313a96eed41282,...,5ead3bea1f313a96eed40564,5ead3bea1f313a96eed40441,5ead3bea1f313a96eed41ae6,5ead3bea1f313a96eed41252,5ead3bea1f313a96eed402ee,5ead3bea1f313a96eed41d3e,5ead3bea1f313a96eed4017b,5ead3bea1f313a96eed41502,5ead3bea1f313a96eed414c4,5ead3bea1f313a96eed41999
5ead3bea1f313a96eed41c32,1.000000,0.149520,0.075059,0.269137,0.243182,0.208738,0.029670,0.155386,0.132686,0.227928,...,0.212771,0.150117,0.266381,0.048450,0.216302,0.197492,0.259665,0.032300,0.337068,0.402569
5ead3bea1f313a96eed414f4,0.149520,1.000000,0.039841,0.238095,0.417029,0.240864,0.062994,0.137464,0.140859,0.193574,...,0.271052,0.079682,0.184428,0.102869,0.283654,0.209657,0.220527,0.154303,0.130120,0.279431
5ead3bea1f313a96eed41ab4,0.075059,0.039841,1.000000,0.039841,0.074767,0.145095,0.000000,0.138013,0.000000,0.121466,...,0.000000,0.100000,0.092582,0.000000,0.033903,0.087706,0.276759,0.129099,0.000000,0.020628
5ead3bea1f313a96eed41bd4,0.269137,0.238095,0.039841,1.000000,0.327666,0.289037,0.062994,0.109971,0.140859,0.258098,...,0.331286,0.159364,0.270494,0.000000,0.310669,0.139771,0.275659,0.034290,0.292770,0.378054
5ead3bea1f313a96eed412d9,0.243182,0.417029,0.074767,0.327666,1.000000,0.325449,0.059108,0.171980,0.070491,0.363266,...,0.433307,0.174456,0.261500,0.193047,0.380224,0.240441,0.241411,0.064349,0.264536,0.370157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ead3bea1f313a96eed41d3e,0.197492,0.209657,0.087706,0.139771,0.240441,0.212095,0.000000,0.060523,0.062017,0.248577,...,0.033150,0.175412,0.135333,0.000000,0.118940,1.000000,0.283190,0.037743,0.214834,0.199016
5ead3bea1f313a96eed4017b,0.259665,0.220527,0.276759,0.275659,0.241411,0.156164,0.036466,0.063661,0.065233,0.261465,...,0.156908,0.092253,0.227759,0.000000,0.125107,0.283190,1.000000,0.079399,0.301297,0.314001
5ead3bea1f313a96eed41502,0.032300,0.154303,0.129099,0.034290,0.064349,0.114472,0.000000,0.207870,0.060858,0.087118,...,0.032530,0.000000,0.172644,0.000000,0.043769,0.037743,0.079399,1.000000,0.000000,0.079894
5ead3bea1f313a96eed414c4,0.337068,0.130120,0.000000,0.292770,0.264536,0.118470,0.000000,0.056344,0.115470,0.165295,...,0.246885,0.244949,0.302372,0.000000,0.193773,0.214834,0.301297,0.000000,1.000000,0.437919


In [86]:
def get3closest(sim_df,user):#user is an ObjectId
    col=sim_df[user].sort_values(ascending=False)[1:]
    return list(col[0:3].index)

get3closest(sim_df,ObjectId('5ead3bea1f313a96eed41502'))

[ObjectId('5ead3bea1f313a96eed419d4'),
 ObjectId('5ead3bea1f313a96eed41045'),
 ObjectId('5ead3bea1f313a96eed41d14')]

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
set(stopwords.words('english'))

In [92]:
get=list(db.chats.find({},{'_id':1}))
    

In [95]:
for g in get:
    print(g)

{'_id': ObjectId('5ead3bec1f313a96eed41de6')}
{'_id': ObjectId('5ead3bec1f313a96eed41de7')}
{'_id': ObjectId('5ead3bec1f313a96eed41de8')}
{'_id': ObjectId('5ead3bec1f313a96eed41de9')}
{'_id': ObjectId('5ead3bec1f313a96eed41dea')}
{'_id': ObjectId('5ead3bec1f313a96eed41deb')}
{'_id': ObjectId('5ead3bec1f313a96eed41dec')}
{'_id': ObjectId('5ead3bec1f313a96eed41ded')}
{'_id': ObjectId('5ead3bec1f313a96eed41dee')}


In [105]:
r = requests.get(f'http://localhost:3500//chat/ids')

In [109]:
list(r.json().keys())

['5ead3bec1f313a96eed41de6',
 '5ead3bec1f313a96eed41de7',
 '5ead3bec1f313a96eed41de8',
 '5ead3bec1f313a96eed41de9',
 '5ead3bec1f313a96eed41dea',
 '5ead3bec1f313a96eed41deb',
 '5ead3bec1f313a96eed41dec',
 '5ead3bec1f313a96eed41ded',
 '5ead3bec1f313a96eed41dee']